In [1]:
from qiskit import QuantumCircuit, QuantumRegister, AncillaRegister
from qiskit.circuit.library import MCPhaseGate, QFTGate 
from math import log2, ceil, pi, gcd

In [2]:
def draper_adder(a, num_of_qubits, num_of_controls, exclude_qft=False):
    ## First num_of_controls qubits are control qubits
    circuit = QuantumCircuit(num_of_qubits + num_of_controls, name=f"Add {a} ({num_of_controls}c+{num_of_qubits}q)")

    qft = None
    if not exclude_qft:
        qft = QFTGate(num_of_qubits)
        qft_indices = range(num_of_controls, num_of_qubits+num_of_controls)
        circuit.append(qft, qft_indices)

    for i in range(num_of_qubits):
        current_phase = 2 * pi * a / (2**(num_of_qubits - i))
        if num_of_controls > 0:
            circuit.mcp(current_phase, list(range(0, num_of_controls)), i+num_of_controls)
        else:
            circuit.p(current_phase, i)

    if not exclude_qft:
        circuit.append(qft.inverse(), qft_indices)

    return circuit.to_gate()


In [3]:
def modular_adder(a, N, num_of_qubits):
    num_of_controls = 2
    circuit = QuantumCircuit(num_of_qubits 
                             + 1 #ancilla qubit
                             + 1 #overflow qubit
                             + num_of_controls, name=f"Modular add {a} mod {N} ({num_of_qubits}q)")

    a_adder = draper_adder(a, num_of_qubits + 1, num_of_controls, exclude_qft=True)
    circuit.append(a_adder, range(0, num_of_controls + num_of_qubits + 1))
    minus_N_adder = draper_adder(-N, num_of_qubits + 1, 0, exclude_qft=True)
    circuit.append(minus_N_adder, range(num_of_controls, num_of_controls + num_of_qubits + 1))
    
    # Perform inverse Quantum Fourier Transform in order to read overflow bit
    qft = QFTGate(num_of_qubits + 1)
    qft_indices = range(num_of_controls, num_of_qubits+num_of_controls+1)
    circuit.append(qft.inverse(), qft_indices)

    overflow_bit_index = num_of_controls + num_of_qubits
    ancilla_bit_index = overflow_bit_index + 1
    # Use the overflow bit to check if we need to add N-a (the inverse of adding a-N)
    circuit.cx(overflow_bit_index, ancilla_bit_index)

    circuit.append(qft, qft_indices)
    # add N back if we weren't greater than N
    N_adder_with_control = draper_adder(N, num_of_qubits + 1, num_of_controls=1, exclude_qft=True)

    # restore ancilla - this is no Joke
    circuit.append(N_adder_with_control, [ancilla_bit_index] + list(qft_indices))
    circuit.append(a_adder.inverse(), range(0, num_of_controls + num_of_qubits + 1))
    circuit.append(qft.inverse(), qft_indices)
    circuit.x(overflow_bit_index)
    circuit.cx(overflow_bit_index, ancilla_bit_index)
    circuit.x(overflow_bit_index)
    circuit.append(qft, qft_indices)
    circuit.append(a_adder, range(0, num_of_controls + num_of_qubits + 1))

    return circuit.to_gate()

In [4]:
def multi_swap(num_of_qubits):
    circuit = QuantumCircuit(2*num_of_qubits) #two copies that will be swapped
    for i in range(num_of_qubits):
        circuit.cx(i, num_of_qubits + i)
        circuit.cx(num_of_qubits + i, i)
        circuit.cx(i, num_of_qubits + i)
    
    return circuit.to_gate()

In [5]:
def inner_modular_multiplier(a, N, num_of_qubits):
    circuit = QuantumCircuit(1 #control                            
                             + num_of_qubits #x input
                             + num_of_qubits #b input
                             + 1 # overflow
                             + 1, # ancilla 
                             name=f"Non-clever modular multiply {a} mod {N} ({num_of_qubits}q)")
    
    control_qubit_index = 0
    overflow_qubit_index = 2*num_of_qubits + 1
    ancilla_qubit_index = 2*num_of_qubits + 2
    b_indices = list(range(num_of_qubits + control_qubit_index + 1, 2*num_of_qubits+control_qubit_index+1)) + [overflow_qubit_index]

    qft = QFTGate(num_of_qubits + 1)
    circuit.append(qft, b_indices)

    for i in range(0, num_of_qubits):
        current_modular_adder = modular_adder((2**i * a) % N, N, num_of_qubits)
        adder_qubits = [control_qubit_index, control_qubit_index + i + 1] + b_indices + [ancilla_qubit_index]
        circuit.append(current_modular_adder, adder_qubits)

    circuit.append(qft.inverse(), b_indices)

    return circuit.to_gate()

In [ ]:
def modular_multiplier(a, N, num_of_qubits = None):
    circuit = QuantumCircuit(1 #control                            
                             + num_of_qubits #x input
                             + num_of_qubits #b input
                             + 1 # overflow
                             + 1, # ancilla 
                             name=f"Modular multiply {a} mod {N} ({num_of_qubits}q)")
    if a <= 0 or N <= 0:
        raise ValueError("Expected a and N to be positive integers!")
    
    if a >= N:
        raise ValueError("Expected a to be smaller than N!")        
    
    if num_of_qubits == None:
        num_of_qubits = ceil(log2(N))

    circuit_qubits = range(0, 2*num_of_qubits + 3)

    modular_multiply_by_a = inner_modular_multiplier(a, N, num_of_qubits)

    circuit.append(modular_multiply_by_a, circuit_qubits)
    swap = multi_swap(num_of_qubits)
    circuit.append(swap, list(range(1, num_of_qubits+1)) + list(range(num_of_qubits+1, 2*num_of_qubits + 1)))
    
    # Clever trick to reset the 0-qubits to zero if a and n are coprime (Figure 7)
    if gcd(a, N) == 1:
        a_inverse = pow(a, -1, N)
        modular_subtract_multiply_by_a = inner_modular_multiplier(a_inverse, N, num_of_qubits).inverse()
        circuit.append(modular_subtract_multiply_by_a, circuit_qubits)

    return circuit.to_gate()

In [7]:
#ran before adding swap
from qiskit.quantum_info import Statevector

myCircuit = QuantumCircuit(11)
myCircuit.x(0) # set control to 1
myCircuit.x(1) # y = 1
myCircuit.x(2) # y = 3
myMultiplier = modular_multiplier(3,5,4) # not sure why 4 instead of 3
myCircuit.append(myMultiplier, range(0, 11))
myCircuit.draw()
 
state = Statevector.from_instruction(myCircuit)
state.probabilities_dict()

{np.str_('00000000001'): np.float64(3.824533025117462e-62),
 np.str_('00000000011'): np.float64(5.5994696061574616e-61),
 np.str_('00000000101'): np.float64(1.8411297835459264e-60),
 np.str_('00000000111'): np.float64(5.373479824551046e-61),
 np.str_('00000001001'): np.float64(0.9999999999999902),
 np.str_('00000001011'): np.float64(5.285737825592983e-61),
 np.str_('00000001101'): np.float64(1.5583828576704402e-61),
 np.str_('00000001111'): np.float64(1.3385140402517768e-61),
 np.str_('00000010001'): np.float64(1.4172520543198936e-61),
 np.str_('00000010011'): np.float64(4.613505967608889e-31),
 np.str_('00000010101'): np.float64(7.995635132362045e-62),
 np.str_('00000010111'): np.float64(2.513089878696508e-62),
 np.str_('00000011001'): np.float64(8.919026517455195e-61),
 np.str_('00000011011'): np.float64(2.8290641418910925e-63),
 np.str_('00000011101'): np.float64(5.684301129911492e-31),
 np.str_('00000011111'): np.float64(1.8576672213000524e-62),
 np.str_('00000100001'): np.float64(

In [8]:
#ran after adding swap
from qiskit.quantum_info import Statevector

myCircuit = QuantumCircuit(11)
myCircuit.x(0) # set control to 1
myCircuit.x(1) # y = 1
myCircuit.x(2) # y = 3
myMultiplier = modular_multiplier(3,5,4) # not sure why 4 instead of 3
myCircuit.append(myMultiplier, range(0, 11))
myCircuit.draw()
 
state = Statevector.from_instruction(myCircuit)
state.probabilities_dict()

{np.str_('00000000001'): np.float64(3.824533025117462e-62),
 np.str_('00000000011'): np.float64(5.5994696061574616e-61),
 np.str_('00000000101'): np.float64(1.8411297835459264e-60),
 np.str_('00000000111'): np.float64(5.373479824551046e-61),
 np.str_('00000001001'): np.float64(0.9999999999999902),
 np.str_('00000001011'): np.float64(5.285737825592983e-61),
 np.str_('00000001101'): np.float64(1.5583828576704402e-61),
 np.str_('00000001111'): np.float64(1.3385140402517768e-61),
 np.str_('00000010001'): np.float64(1.4172520543198936e-61),
 np.str_('00000010011'): np.float64(4.613505967608889e-31),
 np.str_('00000010101'): np.float64(7.995635132362045e-62),
 np.str_('00000010111'): np.float64(2.513089878696508e-62),
 np.str_('00000011001'): np.float64(8.919026517455195e-61),
 np.str_('00000011011'): np.float64(2.8290641418910925e-63),
 np.str_('00000011101'): np.float64(5.684301129911492e-31),
 np.str_('00000011111'): np.float64(1.8576672213000524e-62),
 np.str_('00000100001'): np.float64(

In [9]:
#setting using ceil(log2(N)), 3*4 = 7 (mod 5)!!
from qiskit.quantum_info import Statevector

myMultiplier = modular_multiplier(3,5,ceil(log2(5)))
num_total_qubits = myMultiplier.num_qubits
myCircuit = QuantumCircuit(num_total_qubits)
myCircuit.x(0) # set control to 1
myCircuit.x(3) # y = 4

myCircuit.append(myMultiplier, range(num_total_qubits))
myCircuit.draw()
 
state = Statevector.from_instruction(myCircuit)
my_dict = state.probabilities_dict()
max(my_dict, key=my_dict.get)

np.str_('000000101')

In [10]:
#setting using ceil(log2(N)) + 1, 3*4 = 2
from qiskit.quantum_info import Statevector

myMultiplier = modular_multiplier(3,5,ceil(log2(5)) + 1)
num_total_qubits = myMultiplier.num_qubits
myCircuit = QuantumCircuit(num_total_qubits)
myCircuit.x(0) # set control to 1
myCircuit.x(3) # y = 4

myCircuit.append(myMultiplier, range(num_total_qubits))
myCircuit.draw()
 
state = Statevector.from_instruction(myCircuit)
my_dict = state.probabilities_dict()
max(my_dict, key=my_dict.get)

np.str_('00000000101')

In [11]:
#setting using ceil(log2(N)) + 1, 4*4 = 6
from qiskit.quantum_info import Statevector

myMultiplier = modular_multiplier(4,5,ceil(log2(5)) + 1)
num_total_qubits = myMultiplier.num_qubits
myCircuit = QuantumCircuit(num_total_qubits)
myCircuit.x(0) # set control to 1
myCircuit.x(3) # y = 4

myCircuit.append(myMultiplier, range(num_total_qubits))
myCircuit.draw()
 
state = Statevector.from_instruction(myCircuit)
my_dict = state.probabilities_dict()
max(my_dict, key=my_dict.get)

np.str_('00000000011')

In [12]:
#setting using ceil(log2(N)) + 2, 4*4 = 6
from qiskit.quantum_info import Statevector

myMultiplier = modular_multiplier(4,5,ceil(log2(5)) + 2)
num_total_qubits = myMultiplier.num_qubits
myCircuit = QuantumCircuit(num_total_qubits)
myCircuit.x(0) # set control to 1
myCircuit.x(3) # y = 4

myCircuit.append(myMultiplier, range(num_total_qubits))
myCircuit.draw()
 
state = Statevector.from_instruction(myCircuit)
my_dict = state.probabilities_dict()
max(my_dict, key=my_dict.get)

np.str_('0000000000011')

In [13]:
#setting using ceil(log2(N)) + 3, 4*4 = 1
from qiskit.quantum_info import Statevector

myMultiplier = modular_multiplier(4,5,ceil(log2(5)) + 3)
num_total_qubits = myMultiplier.num_qubits
myCircuit = QuantumCircuit(num_total_qubits)
myCircuit.x(0) # set control to 1
myCircuit.x(3) # y = 4

myCircuit.append(myMultiplier, range(num_total_qubits))
myCircuit.draw()
 
state = Statevector.from_instruction(myCircuit)
my_dict = state.probabilities_dict()
max(my_dict, key=my_dict.get)

np.str_('000000000000011')